#Random Forest - Social Network Ads

Source - https://www.kaggle.com/datasets/rakeshrau/social-network-ads

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


%matplotlib inline

In [ ]:
import pickle as pkl

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, PredefinedSplit, GridSearchCV
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

In [ ]:
# Import Dataset
data = pd.read_csv('drive/My Drive/Colab Notebooks/Titanic_Survival.csv')
data.head(2)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
# Null entries?
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
# Converting categorical features into one-got encoded features

dt_dummy = pd.get_dummies(data[['Pclass','Sex','SibSp','Parch']])
dt_dummy.head(2)

,Pclass,SibSp,Parch,Sex_female,Sex_male
0,3,1,0,0,1
1,1,1,0,1,0


In [ ]:
dt_dummy.isnull().sum()

Pclass        0
SibSp         0
Parch         0
Sex_female    0
Sex_male      0
dtype: int64

In [ ]:
X = dt_dummy
y = data['Survived']

In [ ]:
# Split Train and Test dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)
X_tr, X_val, y_tr, y_val = train_test_split(X_train,y_train, test_size=0.3, random_state=42)


In [ ]:
# Determining the hyperparameters
cv_params = {'n_estimators' : [50,100],
              'max_depth' : [10,50],
              'min_samples_leaf' : [0.5,1],
              'min_samples_split' : [0.001, 0.01],
              'max_features' : ["sqrt"],
              'max_samples' : [.5,.9]
             }

In [ ]:
# Creating list of split indices
split_index = [0 if x in X_val.index else -1 for x in X_train.index]
custom_split = PredefinedSplit(split_index)

In [ ]:
# Building the Model
rf = RandomForestClassifier(random_state=42)

In [ ]:
rf_val = GridSearchCV(rf, cv_params, cv=custom_split, refit='f1',n_jobs=-1, verbose =1)

In [ ]:
%%time
#Fit the model
rf_val.fit(X_train, y_train)

Fitting 1 folds for each of 32 candidates, totalling 32 fits
CPU times: user 228 ms, sys: 52.2 ms, total: 281 ms
Wall time: 5.79 s


GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ..., -1, -1])),
             estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [10, 50], 'max_features': ['sqrt'],
                         'max_samples': [0.5, 0.9],
                         'min_samples_leaf': [0.5, 1],
                         'min_samples_split': [0.001, 0.01],
                         'n_estimators': [50, 100]},
             refit='f1', verbose=1)

In [ ]:
rf_val.best_params_

{'max_depth': 10,
 'max_features': 'sqrt',
 'max_samples': 0.5,
 'min_samples_leaf': 1,
 'min_samples_split': 0.001,
 'n_estimators': 50}

In [ ]:
# Result and Evaluation
# The optimal/best params rf model
rf_opt = RandomForestClassifier(n_estimators=50,
                                max_depth= 10,
                                max_features = 'sqrt',
                                max_samples = 0.5,
                                min_samples_leaf = 1,
                                min_samples_split = 0.001,
                                random_state = 42
                                )

In [ ]:
rf_opt.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, max_samples=0.5, min_samples_split=0.001,
                       n_estimators=50, random_state=42)

In [ ]:
# Predicting using the optimal model
y_pred = rf_opt.predict(X_test)

In [ ]:
# Scores

print('Recall score =', recall_score(y_test, y_pred))
print('Accuracy Score =', accuracy_score(y_test, y_pred ))
print('Precision score =', precision_score(y_test,y_pred))
print('f1 score = ', f1_score(y_test,y_pred))

Recall score = 0.6666666666666666
Accuracy Score = 0.7947761194029851
Precision score = 0.8043478260869565
f1 score =  0.729064039408867
